<a href="https://colab.research.google.com/github/MohammedAyaan/Major-Project---Covid19Analysis/blob/main/Covid19Analysis_and_FbProphet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COVID 19 PANDEMIC ANALYSIS**

**Covid - 19 analysis for India and Prediction using FbProphet**

In [111]:
# Importing necessary libraries 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import plotly.express as px

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from plotly.offline import iplot, init_notebook_mode

In [112]:
India_cases = pd.read_csv('/content/drive/MyDrive/Covid-19 Datasets/Indian_states.csv')
India_cases.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


In [113]:
India_cases.isnull().sum()

Sno                         0
Date                        0
Time                        0
State/UnionTerritory        0
ConfirmedIndianNational     0
ConfirmedForeignNational    0
Cured                       0
Deaths                      0
Confirmed                   0
dtype: int64

In [114]:
India_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15554 entries, 0 to 15553
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       15554 non-null  int64 
 1   Date                      15554 non-null  object
 2   Time                      15554 non-null  object
 3   State/UnionTerritory      15554 non-null  object
 4   ConfirmedIndianNational   15554 non-null  object
 5   ConfirmedForeignNational  15554 non-null  object
 6   Cured                     15554 non-null  int64 
 7   Deaths                    15554 non-null  int64 
 8   Confirmed                 15554 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.1+ MB


In [115]:
India_cases = India_cases.drop(['Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
India_cases.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-01-30,6:00 PM,Kerala,0,0,1
1,2020-01-31,6:00 PM,Kerala,0,0,1
2,2020-02-01,6:00 PM,Kerala,0,0,2
3,2020-02-02,6:00 PM,Kerala,0,0,3
4,2020-02-03,6:00 PM,Kerala,0,0,3


In [116]:
India_cases['Active'] = India_cases['Confirmed'] - India_cases['Cured'] - India_cases['Deaths']
India_cases.tail()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active
15549,2021-06-01,8:00 AM,Telangana,540986,3281,578351,34084
15550,2021-06-01,8:00 AM,Tripura,44908,519,51974,6547
15551,2021-06-01,8:00 AM,Uttarakhand,294671,6452,329494,28371
15552,2021-06-01,8:00 AM,Uttar Pradesh,1633947,20497,1691488,37044
15553,2021-06-01,8:00 AM,West Bengal,1273788,15541,1376377,87048


In [117]:
India_cases['Date'] = pd.to_datetime(India_cases['Date'])

In [118]:
India_cases.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active
0,2020-01-30,6:00 PM,Kerala,0,0,1,1
1,2020-01-31,6:00 PM,Kerala,0,0,1,1
2,2020-02-01,6:00 PM,Kerala,0,0,2,2
3,2020-02-02,6:00 PM,Kerala,0,0,3,3
4,2020-02-03,6:00 PM,Kerala,0,0,3,3


In [119]:
India_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15554 entries, 0 to 15553
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  15554 non-null  datetime64[ns]
 1   Time                  15554 non-null  object        
 2   State/UnionTerritory  15554 non-null  object        
 3   Cured                 15554 non-null  int64         
 4   Deaths                15554 non-null  int64         
 5   Confirmed             15554 non-null  int64         
 6   Active                15554 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 850.7+ KB


In [120]:
india_cases = India_cases[India_cases['Date'] == India_cases['Date'].max()].copy().fillna(0)
india_cases.index = india_cases["State/UnionTerritory"]
india_cases = india_cases.drop(['State/UnionTerritory', 'Time','Date'], axis=1)

In [121]:
india_cases.head()

,Cured,Deaths,Confirmed,Active
State/UnionTerritory,,,,
Andaman and Nicobar Islands,6719,115,7005,171
Andhra Pradesh,1528360,10930,1693085,153795
Arunachal Pradesh,23402,115,27272,3755
Assam,354810,3365,411216,53041
Bihar,685362,5163,706761,16236


Total count as of -> 1 - June - 2021

In [122]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum())).transpose()
df.style.background_gradient(cmap='BuGn',axis=1)

,Cured,Deaths,Confirmed,Active
0,25947629,331895,28175044,1895520


In [123]:
Trend = India_cases.groupby(['Date'])['Confirmed', 'Deaths','Cured',].sum().reset_index()

In [124]:
Trend.head()

,Date,Confirmed,Deaths,Cured
0,2020-01-30,1,0,0
1,2020-01-31,1,0,0
2,2020-02-01,2,0,0
3,2020-02-02,3,0,0
4,2020-02-03,3,0,0


In [125]:
fig = go.Figure(go.Bar(x= Trend.Date, y= Trend.Cured, name='Recovered'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Deaths, name='Deaths'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Confirmed, name='Confirmed'))

fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()

In [126]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [127]:
top_10_death_states = india_cases.sort_values('Deaths',ascending = False)[:10]
horizontal_bar_chart(top_10_death_states,top_10_death_states.Deaths, top_10_death_states.index,
                     'Top 10 States with most deaths','Number of deaths(In Thousands)','State Name','Orange')

State with most deaths is Maharashtra

In [128]:
bottom_5_death_states = india_cases.sort_values('Deaths',ascending = True)[:5]
horizontal_bar_chart(bottom_5_death_states,bottom_5_death_states.Deaths, bottom_5_death_states.index,
                     'Bottom 5 States with most deaths','Number of deaths(In Thousands)','State Name','Orange')

State with least deaths is Dadra and Nagar Haveli and Daman and Diu

In [129]:
top_10_confirmed_states = india_cases.sort_values('Confirmed', ascending=False)[:10]
horizontal_bar_chart(top_10_confirmed_states,top_10_confirmed_states.Confirmed, top_10_confirmed_states.index,
            'Top 10 Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','blue')

State with most confirmed cases is Maharashtra

In [130]:
bottom_5_confirmed_states = india_cases.sort_values('Confirmed', ascending=True)[:5]
horizontal_bar_chart(bottom_5_confirmed_states,bottom_5_confirmed_states.Confirmed, bottom_5_confirmed_states.index,
            'Bottom Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','blue')

State with least confirmed cases is Andaman and Nicobar Islands

In [131]:
top_10_recovered_states = india_cases.sort_values('Cured', ascending=False)[:10]
horizontal_bar_chart(top_10_recovered_states, top_10_recovered_states.Cured, top_10_recovered_states.index,
                    'Top 10 States (Cured Cases)', 'Number of Cured cases (in Thousands)', 'States Name', 'Purple')

State with most cured cases is Maharashtra

In [132]:
bottom_5_recovered_states = india_cases.sort_values('Cured', ascending=True)[:5]
horizontal_bar_chart(bottom_5_recovered_states, bottom_5_recovered_states.Cured, bottom_5_recovered_states.index,
                    'Bottom 5 States (Cured Cases)', 'Number of Cured cases (in Thousands)', 'States Name', 'Purple')

State with least cured cases is Lakshadweep

**Vaccination Status of Indian States(Karnataka)**

In [133]:
vaccination = pd.read_csv(r"/content/drive/MyDrive/Covid-19 Datasets/Indian_states_vaccination.csv")

In [134]:
vaccination.tail()

,Updated On,State,Total Individuals Vaccinated,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,Total Sputnik V Administered,AEFI,18-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered
5139,30/05/2021,West Bengal,10547015.0,381157.0,1170.0,10547015.0,3865954.0,5871662.0,4674095.0,1258.0,1625834.0,12787135.0,0.0,1100.0,2202804.0,4404252.0,3938023.0,14412969.0
5140,31/05/2021,West Bengal,10838457.0,839415.0,2511.0,10838457.0,3879678.0,6050228.0,4786796.0,1433.0,1650999.0,13067136.0,0.0,1144.0,2402085.0,4469534.0,3964698.0,14718135.0
5141,01/06/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5142,02/06/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5143,03/06/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
vaccination['Total Vaccinatons'] = vaccination['First Dose Administered']+vaccination['Second Dose Administered']

#Renaming columns
vaccination.rename(columns = {'Updated On':'Date'}, inplace = True)

In [136]:
Karnataka = vaccination[vaccination["State"]=="Karnataka"]
fig = px.line(Karnataka,x="Date",y="Total Vaccinatons",title="Vaccination till date in Karnataka")  
fig.update_xaxes(rangeslider_visible=True)

In [137]:
fig = px.line(Karnataka,x="Date",y="Total CoviShield Administered",title="CoviShield Administered in Karnataka")  
fig.update_xaxes(rangeslider_visible=True)

In [138]:
fig = px.line(Karnataka,x="Date",y="Total Covaxin Administered",title="Covaxin Administered in Karnataka")  
fig.update_xaxes(rangeslider_visible=True)

Prediction for 30 days using FbProphet

In [139]:
model = Prophet()

In [140]:
Confirmed = Trend.loc[:, ['Date', 'Confirmed']] 
Confirmed.tail()

,Date,Confirmed
484,2021-05-28,27555457
485,2021-05-29,27729247
486,2021-05-30,27894800
487,2021-05-31,28047534
488,2021-06-01,28175044


In [141]:
Cured = Trend.loc[:, ['Date', 'Cured']] 
Cured.tail()

,Date,Cured
484,2021-05-28,24893410
485,2021-05-29,25178011
486,2021-05-30,25454320
487,2021-05-31,25692342
488,2021-06-01,25947629


Prediction for Confirmed cases

In [142]:
Confirmed.columns = ['ds', 'y']
model.fit(Confirmed)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [143]:
future = model.make_future_dataframe(periods=30)
future.tail()

,ds
514,2021-06-27
515,2021-06-28
516,2021-06-29
517,2021-06-30
518,2021-07-01


In [144]:
forecast_india_conf = model.predict(future)
forecast_india_conf

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-1.888260e+05,-1.563324e+06,1.088135e+06,-1.888260e+05,-1.888260e+05,-14631.885644,-14631.885644,-14631.885644,-14631.885644,-14631.885644,-14631.885644,0.0,0.0,0.0,-2.034579e+05
1,2020-01-31,-1.856222e+05,-1.620839e+06,1.018437e+06,-1.856222e+05,-1.856222e+05,-6748.513905,-6748.513905,-6748.513905,-6748.513905,-6748.513905,-6748.513905,0.0,0.0,0.0,-1.923707e+05
2,2020-02-01,-1.824184e+05,-1.494507e+06,1.198459e+06,-1.824184e+05,-1.824184e+05,1996.620514,1996.620514,1996.620514,1996.620514,1996.620514,1996.620514,0.0,0.0,0.0,-1.804218e+05
3,2020-02-02,-1.792146e+05,-1.523848e+06,1.297928e+06,-1.792146e+05,-1.792146e+05,10531.991187,10531.991187,10531.991187,10531.991187,10531.991187,10531.991187,0.0,0.0,0.0,-1.686826e+05
4,2020-02-03,-1.760108e+05,-1.505300e+06,1.246737e+06,-1.760108e+05,-1.760108e+05,16767.594822,16767.594822,16767.594822,16767.594822,16767.594822,16767.594822,0.0,0.0,0.0,-1.592432e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,2021-06-27,2.911384e+07,2.762092e+07,3.063506e+07,2.884969e+07,2.945366e+07,10531.991187,10531.991187,10531.991187,10531.991187,10531.991187,10531.991187,0.0,0.0,0.0,2.912438e+07
515,2021-06-28,2.927465e+07,2.793024e+07,3.063178e+07,2.898717e+07,2.963714e+07,16767.594822,16767.594822,16767.594822,16767.594822,16767.594822,16767.594822,0.0,0.0,0.0,2.929142e+07
516,2021-06-29,2.943546e+07,2.795189e+07,3.086037e+07,2.912278e+07,2.981668e+07,17177.805997,17177.805997,17177.805997,17177.805997,17177.805997,17177.805997,0.0,0.0,0.0,2.945264e+07
517,2021-06-30,2.959627e+07,2.811743e+07,3.103550e+07,2.926233e+07,2.999273e+07,-25093.612970,-25093.612970,-25093.612970,-25093.612970,-25093.612970,-25093.612970,0.0,0.0,0.0,2.957118e+07


In [145]:
fig = plot_plotly(model, forecast_india_conf) 

fig.update_layout(template='plotly_white')

iplot(fig)

Prediction for Recovered cases

In [146]:
model1 = Prophet()

In [147]:
Cured.columns = ['ds', 'y']
model1.fit(Cured)

future_1 = model1.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
future_1.tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds
544,2021-07-27
545,2021-07-28
546,2021-07-29
547,2021-07-30
548,2021-07-31


In [148]:
forecast_india_cured = model1.predict(future_1)
forecast_india_cured

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-1.597850e+05,-1.498357e+06,1.204747e+06,-1.597850e+05,-1.597850e+05,-19965.599927,-19965.599927,-19965.599927,-19965.599927,-19965.599927,-19965.599927,0.0,0.0,0.0,-1.797506e+05
1,2020-01-31,-1.572614e+05,-1.427534e+06,1.191967e+06,-1.572614e+05,-1.572614e+05,-11085.287132,-11085.287132,-11085.287132,-11085.287132,-11085.287132,-11085.287132,0.0,0.0,0.0,-1.683467e+05
2,2020-02-01,-1.547378e+05,-1.469700e+06,1.217265e+06,-1.547378e+05,-1.547378e+05,-777.561758,-777.561758,-777.561758,-777.561758,-777.561758,-777.561758,0.0,0.0,0.0,-1.555154e+05
3,2020-02-02,-1.522142e+05,-1.605157e+06,1.160539e+06,-1.522142e+05,-1.522142e+05,11052.013260,11052.013260,11052.013260,11052.013260,11052.013260,11052.013260,0.0,0.0,0.0,-1.411622e+05
4,2020-02-03,-1.496906e+05,-1.512988e+06,1.297586e+06,-1.496906e+05,-1.496906e+05,19544.089787,19544.089787,19544.089787,19544.089787,19544.089787,19544.089787,0.0,0.0,0.0,-1.301465e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,2021-07-27,2.764657e+07,2.599940e+07,2.922587e+07,2.668069e+07,2.850008e+07,32080.878139,32080.878139,32080.878139,32080.878139,32080.878139,32080.878139,0.0,0.0,0.0,2.767865e+07
545,2021-07-28,2.776497e+07,2.608276e+07,2.927663e+07,2.676096e+07,2.864234e+07,-30848.532369,-30848.532369,-30848.532369,-30848.532369,-30848.532369,-30848.532369,0.0,0.0,0.0,2.773412e+07
546,2021-07-29,2.788337e+07,2.629980e+07,2.963278e+07,2.685202e+07,2.878555e+07,-19965.599927,-19965.599927,-19965.599927,-19965.599927,-19965.599927,-19965.599927,0.0,0.0,0.0,2.786341e+07
547,2021-07-30,2.800178e+07,2.627560e+07,2.962469e+07,2.693658e+07,2.893180e+07,-11085.287132,-11085.287132,-11085.287132,-11085.287132,-11085.287132,-11085.287132,0.0,0.0,0.0,2.799069e+07


In [149]:
fig = plot_plotly(model1, forecast_india_cured) 

fig.update_layout(template='plotly_white')

iplot(fig)

**Covid - 19 analysis for World and Prediction using FbProphet**

In [150]:
world = pd.read_csv('/content/drive/MyDrive/Covid-19 Datasets/World.csv')
world

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
306424,306425,05/29/2021,Zaporizhia Oblast,Ukraine,2021-05-30 04:20:55,102641.0,2335.0,95289.0
306425,306426,05/29/2021,Zeeland,Netherlands,2021-05-30 04:20:55,29147.0,245.0,0.0
306426,306427,05/29/2021,Zhejiang,Mainland China,2021-05-30 04:20:55,1364.0,1.0,1324.0
306427,306428,05/29/2021,Zhytomyr Oblast,Ukraine,2021-05-30 04:20:55,87550.0,1738.0,83790.0


In [151]:
world.isnull().sum()

SNo                    0
ObservationDate        0
Province/State     78100
Country/Region         0
Last Update            0
Confirmed              0
Deaths                 0
Recovered              0
dtype: int64

In [152]:
world.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306429 entries, 0 to 306428
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SNo              306429 non-null  int64  
 1   ObservationDate  306429 non-null  object 
 2   Province/State   228329 non-null  object 
 3   Country/Region   306429 non-null  object 
 4   Last Update      306429 non-null  object 
 5   Confirmed        306429 non-null  float64
 6   Deaths           306429 non-null  float64
 7   Recovered        306429 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 18.7+ MB


In [153]:
world = world.drop(['SNo','Province/State', 'Last Update'], axis=1)
world.head()

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered
0,01/22/2020,Mainland China,1.0,0.0,0.0
1,01/22/2020,Mainland China,14.0,0.0,0.0
2,01/22/2020,Mainland China,6.0,0.0,0.0
3,01/22/2020,Mainland China,1.0,0.0,0.0
4,01/22/2020,Mainland China,0.0,0.0,0.0


In [154]:
world['Active'] = world['Confirmed'] - world['Recovered'] - world['Deaths']
world.tail()

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered,Active
306424,05/29/2021,Ukraine,102641.0,2335.0,95289.0,5017.0
306425,05/29/2021,Netherlands,29147.0,245.0,0.0,28902.0
306426,05/29/2021,Mainland China,1364.0,1.0,1324.0,39.0
306427,05/29/2021,Ukraine,87550.0,1738.0,83790.0,2022.0
306428,05/29/2021,Netherlands,391559.0,4252.0,0.0,387307.0


In [155]:
world['ObservationDate'] = pd.to_datetime(world['ObservationDate'])

In [156]:
world.tail()

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered,Active
306424,2021-05-29,Ukraine,102641.0,2335.0,95289.0,5017.0
306425,2021-05-29,Netherlands,29147.0,245.0,0.0,28902.0
306426,2021-05-29,Mainland China,1364.0,1.0,1324.0,39.0
306427,2021-05-29,Ukraine,87550.0,1738.0,83790.0,2022.0
306428,2021-05-29,Netherlands,391559.0,4252.0,0.0,387307.0


In [157]:
world.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306429 entries, 0 to 306428
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ObservationDate  306429 non-null  datetime64[ns]
 1   Country/Region   306429 non-null  object        
 2   Confirmed        306429 non-null  float64       
 3   Deaths           306429 non-null  float64       
 4   Recovered        306429 non-null  float64       
 5   Active           306429 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 14.0+ MB


In [158]:
world_cases = world[world['ObservationDate'] == world['ObservationDate'].max()].copy().fillna(0)
world_cases.index = world_cases["Country/Region"]
world_cases = world_cases.drop(['Country/Region', 'ObservationDate'], axis=1)

In [159]:
world_cases.head()

,Confirmed,Deaths,Recovered,Active
Country/Region,,,,
Afghanistan,70111.0,2899.0,57281.0,9931.0
Albania,132297.0,2449.0,129215.0,633.0
Algeria,128456.0,3460.0,89419.0,35577.0
Andorra,13693.0,127.0,13416.0,150.0
Angola,34180.0,757.0,27646.0,5777.0


Total count as of -> 29 - May - 2021

In [160]:
df = pd.DataFrame(pd.to_numeric(world_cases.sum())).transpose()
df.style.background_gradient(cmap='BuGn',axis=1)

,Confirmed,Deaths,Recovered,Active
0,169951560.000000,3533619.000000,107140669.000000,59277272.000000


In [161]:
Trend = world.groupby(['ObservationDate'])['Confirmed', 'Deaths','Recovered',].sum().reset_index()

In [162]:
Trend.head()

,ObservationDate,Confirmed,Deaths,Recovered
0,2020-01-22,557.0,17.0,30.0
1,2020-01-23,1097.0,34.0,60.0
2,2020-01-24,941.0,26.0,39.0
3,2020-01-25,1437.0,42.0,42.0
4,2020-01-26,2118.0,56.0,56.0


In [163]:
fig = go.Figure(go.Bar(x= Trend.ObservationDate, y= Trend.Recovered, name='Recovered'))
fig.add_trace(go.Bar(x=Trend.ObservationDate, y= Trend.Deaths, name='Deaths'))
fig.add_trace(go.Bar(x=Trend.ObservationDate, y= Trend.Confirmed, name='Confirmed'))

fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()

In [164]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [165]:
top_10_death_countries = world_cases.sort_values('Deaths',ascending = False)[:10]
horizontal_bar_chart(top_10_death_countries,top_10_death_countries.Deaths, top_10_death_countries.index,
                     'Top 10 Countries with most deaths','Number of deaths(In Thousands)','Country Name','Orange')

Country with most deaths is US

In [166]:
bottom_5_death_countries = world_cases.sort_values('Deaths',ascending = True)[:50]
horizontal_bar_chart(bottom_5_death_countries,bottom_5_death_countries.Deaths, bottom_5_death_countries.index,
                     'Bottom 5 Countries with most deaths','Number of deaths(In Thousands)','Country Name','Orange')

In [167]:
top_10_confirmed_countries = world_cases.sort_values('Confirmed', ascending=False)[:10]
horizontal_bar_chart(top_10_confirmed_countries,top_10_confirmed_countries.Confirmed, top_10_confirmed_countries.index,
            'Top 10 Countries (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','Country Name','blue')

County with most deaths is US

In [168]:
bottom_10_confirmed_countries = world_cases.sort_values('Confirmed', ascending=True)[:10]
horizontal_bar_chart(bottom_10_confirmed_countries,bottom_10_confirmed_countries.Confirmed, bottom_10_confirmed_countries.index,
            'Bottom 10 Countries (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','Country Name','blue')

In [169]:
top_10_recovered_countries = world_cases.sort_values('Recovered', ascending=False)[:10]
horizontal_bar_chart(top_10_recovered_countries, top_10_recovered_countries.Recovered, top_10_recovered_countries.index,
                    'Top 10 Countries (Recovered Cases)', 'Number of Recovered cases (in Thousands)', 'Countries Name', 'Purple')

County with most recovered cases is India

In [170]:
bottom_10_recovered_countries = world_cases.sort_values('Recovered', ascending=True)[:200]
horizontal_bar_chart(bottom_10_recovered_countries, bottom_10_recovered_countries.Recovered, bottom_10_recovered_countries.index,
                    'Bottom 10 Countries (Recovered Cases)', 'Number of Recovered cases (in Thousands)', 'Countries Name', 'Purple')

**Vaccination Status of World**

In [171]:
world_vaccination = pd.read_csv(r"/content/drive/MyDrive/Covid-19 Datasets/World_vaccination.csv")

In [172]:
world_vaccination.tail()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
21623,Zimbabwe,ZWE,2021-05-27,953389.0,648121.0,305268.0,16349.0,12285.0,6.41,4.36,2.05,827.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1399441311...
21624,Zimbabwe,ZWE,2021-05-28,976796.0,656630.0,320166.0,23407.0,12695.0,6.57,4.42,2.15,854.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1399441311...
21625,Zimbabwe,ZWE,2021-05-29,1002465.0,666786.0,335679.0,25669.0,14056.0,6.74,4.49,2.26,946.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1399441311...
21626,Zimbabwe,ZWE,2021-05-30,1011973.0,670755.0,341218.0,9508.0,14420.0,6.81,4.51,2.30,970.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1399441311...
21627,Zimbabwe,ZWE,2021-05-31,1020078.0,675678.0,344400.0,8105.0,15022.0,6.86,4.55,2.32,1011.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1399441311...


In [173]:
world_vaccination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21628 entries, 0 to 21627
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              21628 non-null  object 
 1   iso_code                             21628 non-null  object 
 2   date                                 21628 non-null  object 
 3   total_vaccinations                   12275 non-null  float64
 4   people_vaccinated                    11479 non-null  float64
 5   people_fully_vaccinated              8967 non-null   float64
 6   daily_vaccinations_raw               10187 non-null  float64
 7   daily_vaccinations                   21412 non-null  float64
 8   total_vaccinations_per_hundred       12275 non-null  float64
 9   people_vaccinated_per_hundred        11479 non-null  float64
 10  people_fully_vaccinated_per_hundred  8967 non-null   float64
 11  daily_vaccinations_per_milli

In [174]:
India = world_vaccination[world_vaccination["country"]=="India"]
fig = px.line(India,x="date",y="total_vaccinations",title="Vaccination till date in India")  
fig.update_xaxes(rangeslider_visible=True)

Prediction of world cases for 30 days using FbProphet

In [175]:
world_model = Prophet()

In [176]:
world_Confirmed = Trend.loc[:, ['ObservationDate', 'Confirmed']] 
world_Confirmed.tail()

,ObservationDate,Confirmed
489,2021-05-25,167848207.0
490,2021-05-26,168416423.0
491,2021-05-27,168970791.0
492,2021-05-28,169470725.0
493,2021-05-29,169951560.0


In [177]:
world_Recovered = Trend.loc[:, ['ObservationDate', 'Recovered']] 
world_Recovered.tail()

,ObservationDate,Recovered
489,2021-05-25,104824935.0
490,2021-05-26,105380748.0
491,2021-05-27,106078106.0
492,2021-05-28,106633069.0
493,2021-05-29,107140669.0


In [178]:
world_Confirmed = world_Confirmed.rename(columns = {'Confirmed':'y','ObservationDate':'ds'})

In [179]:
world_Confirmed.columns = ['ds', 'y']
world_model.fit(world_Confirmed)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [180]:
world_future = world_model.make_future_dataframe(periods=30)
world_future.tail()

,ds
519,2021-06-24
520,2021-06-25
521,2021-06-26
522,2021-06-27
523,2021-06-28


In [181]:
forecast_world_conf = world_model.predict(world_future)
forecast_world_conf

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-22,-1.040022e+05,-1.530411e+06,1.230884e+06,-1.040022e+05,-1.040022e+05,-23962.724205,-23962.724205,-23962.724205,-23962.724205,-23962.724205,-23962.724205,0.0,0.0,0.0,-1.279650e+05
1,2020-01-23,-9.799641e+04,-1.506939e+06,1.290931e+06,-9.799641e+04,-9.799641e+04,25059.228924,25059.228924,25059.228924,25059.228924,25059.228924,25059.228924,0.0,0.0,0.0,-7.293718e+04
2,2020-01-24,-9.199058e+04,-1.271851e+06,1.304787e+06,-9.199058e+04,-9.199058e+04,64033.994789,64033.994789,64033.994789,64033.994789,64033.994789,64033.994789,0.0,0.0,0.0,-2.795659e+04
3,2020-01-25,-8.598476e+04,-1.290689e+06,1.247497e+06,-8.598476e+04,-8.598476e+04,58748.029705,58748.029705,58748.029705,58748.029705,58748.029705,58748.029705,0.0,0.0,0.0,-2.723673e+04
4,2020-01-26,-7.997894e+04,-1.336890e+06,1.188431e+06,-7.997894e+04,-7.997894e+04,-5528.500605,-5528.500605,-5528.500605,-5528.500605,-5528.500605,-5528.500605,0.0,0.0,0.0,-8.550744e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,2021-06-24,1.851228e+08,1.830844e+08,1.871748e+08,1.837061e+08,1.865160e+08,25059.228924,25059.228924,25059.228924,25059.228924,25059.228924,25059.228924,0.0,0.0,0.0,1.851478e+08
520,2021-06-25,1.857478e+08,1.835674e+08,1.878553e+08,1.842152e+08,1.872854e+08,64033.994789,64033.994789,64033.994789,64033.994789,64033.994789,64033.994789,0.0,0.0,0.0,1.858119e+08
521,2021-06-26,1.863729e+08,1.843300e+08,1.886054e+08,1.847566e+08,1.880006e+08,58748.029705,58748.029705,58748.029705,58748.029705,58748.029705,58748.029705,0.0,0.0,0.0,1.864317e+08
522,2021-06-27,1.869980e+08,1.850231e+08,1.891597e+08,1.853126e+08,1.887163e+08,-5528.500605,-5528.500605,-5528.500605,-5528.500605,-5528.500605,-5528.500605,0.0,0.0,0.0,1.869924e+08


Prediction for confirmed cases

In [182]:
world_fig = plot_plotly(world_model, forecast_world_conf) 

world_fig.update_layout(template='plotly_white')

iplot(world_fig)

In [183]:
world_model1 = Prophet()

In [184]:
world_Recovered = world_Recovered.rename(columns = {'Recovered':'y','ObservationDate':'ds'})
world_Recovered.columns = ['ds', 'y']
world_model1.fit(world_Recovered)

world_future_1 = world_model1.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
world_future_1.tail()

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds
549,2021-07-24
550,2021-07-25
551,2021-07-26
552,2021-07-27
553,2021-07-28


In [185]:
forecast_world_recovered = world_model1.predict(world_future_1)
forecast_world_recovered

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-22,-2.579220e+05,-2.127443e+06,1.586229e+06,-2.579220e+05,-2.579220e+05,-18902.287910,-18902.287910,-18902.287910,-18902.287910,-18902.287910,-18902.287910,0.0,0.0,0.0,-2.768243e+05
1,2020-01-23,-2.495010e+05,-2.167479e+06,1.612039e+06,-2.495010e+05,-2.495010e+05,4629.571006,4629.571006,4629.571006,4629.571006,4629.571006,4629.571006,0.0,0.0,0.0,-2.448715e+05
2,2020-01-24,-2.410801e+05,-1.992602e+06,1.707854e+06,-2.410801e+05,-2.410801e+05,18697.093062,18697.093062,18697.093062,18697.093062,18697.093062,18697.093062,0.0,0.0,0.0,-2.223830e+05
3,2020-01-25,-2.326591e+05,-2.033557e+06,1.723996e+06,-2.326591e+05,-2.326591e+05,63739.358541,63739.358541,63739.358541,63739.358541,63739.358541,63739.358541,0.0,0.0,0.0,-1.689198e+05
4,2020-01-26,-2.242381e+05,-2.170898e+06,1.562988e+06,-2.242381e+05,-2.242381e+05,-13720.932419,-13720.932419,-13720.932419,-13720.932419,-13720.932419,-13720.932419,0.0,0.0,0.0,-2.379591e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,2021-07-24,1.256804e+08,1.228909e+08,1.284871e+08,1.235768e+08,1.276712e+08,63739.358541,63739.358541,63739.358541,63739.358541,63739.358541,63739.358541,0.0,0.0,0.0,1.257441e+08
550,2021-07-25,1.261077e+08,1.233051e+08,1.288158e+08,1.239047e+08,1.281626e+08,-13720.932419,-13720.932419,-13720.932419,-13720.932419,-13720.932419,-13720.932419,0.0,0.0,0.0,1.260940e+08
551,2021-07-26,1.265351e+08,1.236136e+08,1.295587e+08,1.242636e+08,1.286621e+08,524.092464,524.092464,524.092464,524.092464,524.092464,524.092464,0.0,0.0,0.0,1.265356e+08
552,2021-07-27,1.269625e+08,1.239888e+08,1.295293e+08,1.245954e+08,1.291549e+08,-54966.894744,-54966.894744,-54966.894744,-54966.894744,-54966.894744,-54966.894744,0.0,0.0,0.0,1.269075e+08


Prediction for recovered cases

In [186]:
world_fig = plot_plotly(world_model1, forecast_world_recovered) 

world_fig.update_layout(template='plotly_white')

iplot(world_fig)

**Major Vaccine Manufactures in the world**

In [187]:
vac_manuf = pd.read_csv("/content/drive/MyDrive/Covid-19 Datasets/vaccine_manufactures.csv")
vac_manuf 

,location,date,vaccine,total_vaccinations
0,Chile,2020-12-24,Pfizer/BioNTech,420
1,Chile,2020-12-25,Pfizer/BioNTech,5198
2,Chile,2020-12-26,Pfizer/BioNTech,8338
3,Chile,2020-12-27,Pfizer/BioNTech,8649
4,Chile,2020-12-28,Pfizer/BioNTech,8649
...,...,...,...,...
4680,United States,2021-05-29,Moderna,124067556
4681,United States,2021-05-29,Pfizer/BioNTech,158708623
4682,United States,2021-05-30,Johnson&Johnson,10776617
4683,United States,2021-05-30,Moderna,124520914


Contry with most vaccinated people is United States

In [188]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [189]:
vac_manuf.isnull().sum()

location              0
date                  0
vaccine               0
total_vaccinations    0
dtype: int64

In [190]:
vac_manuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4685 entries, 0 to 4684
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   location            4685 non-null   object
 1   date                4685 non-null   object
 2   vaccine             4685 non-null   object
 3   total_vaccinations  4685 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 146.5+ KB


In [191]:
vac_manuf['Date'] = pd.to_datetime(vac_manuf['date'])

In [192]:
vac_manuf.head()

,location,date,vaccine,total_vaccinations,Date
0,Chile,2020-12-24,Pfizer/BioNTech,420,2020-12-24
1,Chile,2020-12-25,Pfizer/BioNTech,5198,2020-12-25
2,Chile,2020-12-26,Pfizer/BioNTech,8338,2020-12-26
3,Chile,2020-12-27,Pfizer/BioNTech,8649,2020-12-27
4,Chile,2020-12-28,Pfizer/BioNTech,8649,2020-12-28


In [193]:
vac_manuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4685 entries, 0 to 4684
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   location            4685 non-null   object        
 1   date                4685 non-null   object        
 2   vaccine             4685 non-null   object        
 3   total_vaccinations  4685 non-null   int64         
 4   Date                4685 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 183.1+ KB


In [194]:
Trend = vac_manuf.groupby(['vaccine'])['total_vaccinations'].sum().reset_index()

In [195]:
Trend

,vaccine,total_vaccinations
0,Johnson&Johnson,553935713
1,Moderna,9172305173
2,Oxford/AstraZeneca,1111707894
3,Pfizer/BioNTech,15310054342
4,Sinovac,1007748237


In [196]:
fig = go.Figure(go.Bar(x= Trend.vaccine,y= Trend.total_vaccinations,name='Vaccines'))
fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()